# Tarefa 1: Setup e Consolidação dos Dados
1. Download automatizado:

    * Códigos para baixar arquivos mensais de 2023 e 2024 (meta: 24 meses, mínimo: 12
        meses)

    * Encontrar os dados e o padrão da URL em:
        `https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page`

    * Focar apenas nos Yellow Taxi Trip Records

    * Verificar integridade dos arquivos e tratar erros